In [21]:
import pandas as pd
import requests 

In [22]:
artist_df = pd.read_csv('music.csv')

In [41]:
artist_info_list = []
for index,row in artist_df.iterrows():
    artist_name  = row['artist']
    query_url = f'http://musicbrainz.org/ws/2/artist/?query=name:{artist_name}&inc=aliases&fmt=json'
    response = requests.get(query_url).json()
    artist_info = {
        'artist': artist_name,
        'city': '',
        'started':'',
        'ended':'',
        'desc':'',
        'id':''
    }
    artist = response['artists'][0]
    artist_info['id'] = artist['id']
    try:
        artist_info['city']  = artist['begin-area']['name']
    except:
        pass
    try:
        artist_info['started'] = artist['life-span']['begin']
    except:
        pass
    try:
        artist_info['ended'] = artist['life-span']['end']
    except:
        pass
    try:
        artist_info['desc'] = artist['disambiguation']
    except:
        pass
    artist_info_list.append(artist_info)

In [57]:
artist_df = pd.DataFrame(artist_info_list)

In [73]:
release_list = []
for index,row in artist_df.iterrows():
    release_url = f"https://musicbrainz.org/ws/2/release/?query=arid:{row['id']}&fmt=json"
    response = requests.get(release_url).json()
    releases = response['releases']
    for release in releases:
        try:
            release_dict = {
            'artist': row['artist'],
            'title' : release['title'],
            'tracks' : release['track-count'],
            'date' : release['date']
            }
            release_list.append(release_dict)
        except:
            pass
        

In [76]:
releases_df = pd.DataFrame(release_list)

In [79]:
releases_df = releases_df.set_index('artist')
artist_df = artist_df.set_index('artist')

In [83]:
final_final_df = pd.merge(releases_df,artist_df,left_index=True, right_index=True)

In [84]:
final_final_df.to_csv('artist_info.csv')